In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 86kB/s 
     |████████████████████████████████| 204kB 41.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=8657dee57974333c2e2beb0e01cacff695c80758654ed0c58e3edb2fd9d68920
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cluster').getOrCreate()

from pyspark.ml.clustering import KMeans

dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')

In [ ]:
final_data = dataset.select('features')

In [ ]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [ ]:
kmeans = KMeans().setK(3).setSeed(1)

In [ ]:
model = kmeans.fit(final_data)

In [ ]:
wssse = model.clusterCenters(final_data)

TypeError: ignored

In [ ]:
print(wssse)

In [ ]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [ ]:
centers = model.clusterCenters()

In [ ]:
centers

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [ ]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [ ]:
model.transform(final_data)

DataFrame[features: vector, prediction: int]

In [ ]:
results= model.transform(final_data)

results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+



# Clustering Example Code

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cluster').getOrCreate()

dataset = spark.read.csv('seeds_dataset.csv', inferSchema=True, header=True)

In [ ]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [ ]:
dataset.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [ ]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [ ]:
from pyspark.ml.clustering import KMeans

from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [ ]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [ ]:
final_data = assembler.transform(dataset)

In [ ]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [ ]:
scaler_model = scaler.fit(final_data)

In [ ]:
final_data = scaler_model.transform(final_data)

In [ ]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [ ]:
kmeans = KMeans(featuresCol = 'scaledFeatures', k=3)

In [ ]:
model = kmeans.fit(final_data)

In [ ]:
print('WSSSE')
print(model.computeCost(final_data))

WSSSE


AttributeError: ignored

In [ ]:
centers = model.clusterCenters()

In [ ]:
print(centers)

In [ ]:
model.transform(final_data).show()

In [ ]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
+----------+
only showing top 20 rows



# K-Means Clustering Consulting Project

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cluster').getOrCreate()

dataset = spark.read.csv('hack_data.csv', inferSchema=True, header=True)

In [ ]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [ ]:
from pyspark.ml.clustering import KMeans

from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
feat_cols =['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
assembler = VectorAssembler(inputCols=feat_cols, outputCol='features')

In [ ]:
final_data1 = assembler.transform(dataset)

IllegalArgumentException: ignored

In [ ]:
final_data1.printSchema()

NameError: ignored

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol= 'scaledFeatures')

In [ ]:
scaler_model = scaler.fit(final_data1)

cluster_final_data =scaler_model.transform(final_data1)

NameError: ignored

In [ ]:
cluster_final_data.printSchema()

NameError: ignored

In [ ]:
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=2)

In [ ]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

NameError: ignored

In [ ]:
model_k3.transform(cluster_final_data).select('prediction').show()

NameError: ignored

In [ ]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

NameError: ignored

In [ ]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

NameError: ignored